<a href="https://colab.research.google.com/github/davialvesguerra/trainee-iee/blob/main/5%C2%BA_Per%C3%ADodo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import time
import os

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
path_ = 'gdrive/My Drive/iee/5º Período/'
os.chdir(path_)
os.getcwd()

'/content/gdrive/My Drive/iee/5º Período'

In [ ]:
if os.path.exists('animals/train') == False:
  os.mkdir('animals/train')


if os.path.exists('animals/test') == False:
  os.mkdir('animals/test')

In [ ]:
cats = os.listdir('animals/cats')
cats_src = ['cats/'+x for x in cats]

dogs = os.listdir('animals/dogs')
dogs_src = ['dogs/'+x for x in dogs]

panda = os.listdir('animals/panda')
panda_src = ['panda/'+x for x in panda]

In [ ]:
train = []
test = []

N = 1000
size = 0.7


train_src = cats_src[0:int(N*size)]
train_src += dogs_src[0:int(N*size)]
train_src += panda_src[0:int(N*size)]

test_src = cats_src[int(N*size):]
test_src += dogs_src[int(N*size):]
test_src += panda_src[int(N*size):]

# train = cats[0:int(N*size)]
# train += dogs[0:int(N*size)]
# train += panda[0:int(N*size)]

# test = cats[int(N*size):]
# test += dogs[int(N*size):]
# test += panda[int(N*size):]


In [ ]:
if os.path.exists('animals/train/panda') == False:
  os.mkdir('animals/train/panda')
  os.mkdir('animals/test/panda')

if os.path.exists('animals/train/dogs') == False:
  os.mkdir('animals/train/dogs')
  os.mkdir('animals/test/dogs')

if os.path.exists('animals/train/cats') == False:
  os.mkdir('animals/train/cats')
  os.mkdir('animals/test/cats')


In [ ]:
# import shutil

# for value in train_src:

#   dst = 'animals/train/'+value
#   src = 'animals/'+value
  
#   shutil.copyfile(src, dst)


In [ ]:
# for value in test_src:

#   dst = 'animals/test/'+value
#   src = 'animals/'+value
  
#   shutil.copyfile(src, dst)


In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)


test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
        'animals/train',
        target_size=(150, 150),
        batch_size=32
        )

train_generator

Found 2100 images belonging to 3 classes.


In [ ]:
from tensorflow.keras import layers, models

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))



In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='mse',
              metrics=['accuracy'])

In [ ]:
model.fit(train_generator, epochs=10)

In [ ]:
base_model = tf.keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False)

83697664/83683744 [==============================] - 1s 0us/step


In [ ]:
base_model.trainable = False

In [ ]:
inputs = tf.keras.Input(shape=(150, 150, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = tf.keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = tf.keras.layers.Dense(3)(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.BinaryAccuracy()])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense_2 (Dense)             (None, 3)                 6147      
                                                                 
Total params: 20,867,627
Trainable params: 6,147
Non-trainable params: 20,861,480
_________________________________________________________________


In [ ]:
model.fit(train_generator, epochs=4)

Epoch 1/4
66/66 [==============================] - 215s 3s/step - loss: 0.1917 - binary_accuracy: 0.9116
Epoch 2/4
66/66 [==============================] - 178s 3s/step - loss: 0.0843 - binary_accuracy: 0.9652
Epoch 3/4
66/66 [==============================] - 177s 3s/step - loss: 0.0689 - binary_accuracy: 0.9729
Epoch 4/4
66/66 [==============================] - 176s 3s/step - loss: 0.0619 - binary_accuracy: 0.9756
